<a href="https://colab.research.google.com/github/schokoro/cnn_crf_nertagger/blob/dev/NER_tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN и CRF для извлечения именованных сущностей



In [1]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device == 'cpu':
    print('cpu')
else:
    n_gpu = torch.cuda.device_count()
    print(torch.cuda.get_device_name(0))
    

Tesla T4


In [0]:
!rm -rf cnn_crf_nertagger/

In [3]:
!git clone -b dev https://github.com/schokoro/cnn_crf_nertagger.git > /dev/null
!pip install allennlp wget> /dev/null


Cloning into 'cnn_crf_nertagger'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 106 (delta 44), reused 92 (delta 35), pack-reused 0
Receiving objects: 100% (106/106), 24.58 KiB | 12.29 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [0]:
import sys
sys.path.append('/content/cnn_crf_nertagger')

In [0]:
%load_ext autoreload
%autoreload 2

import spacy
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report
import numpy as np
from torch.nn import functional as F
from torch.utils.data import TensorDataset
from allennlp.data.dataset_readers.conll2003 import Conll2003DatasetReader
from allennlp.common.util import ensure_list
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset
from pdb import set_trace
from gc import collect
import wget
import cnn_crf_nertagger
from cnn_crf_nertagger.modules.modules import NERTaggerModel, NERTagger
from cnn_crf_nertagger.utils.pipeline import train_eval_loop, predict_with_model
from cnn_crf_nertagger.utils.prepare import tag_corpus_to_tensor, tokenize_corpus
from os import path, listdir



# torch.backends.cudnn.deterministic=False 

## Загрузка корпусов

In [0]:
!rm /content/cnn_crf_nertagger/data/*


In [0]:
path_data = '/content/cnn_crf_nertagger/data/'
path_train = '/content/cnn_crf_nertagger/data/eng.train'
path_valid = '/content/cnn_crf_nertagger/data/eng.testa'
path_test = '/content/cnn_crf_nertagger/data/eng.testb'

dataset_urls = {
    'eng.testa': 'https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testa',
    'eng.testb': 'https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb',
    'eng.train': 'https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.train'}
for file_name in dataset_urls:
    wget.download(dataset_urls[file_name], path.join(path_data, file_name))

In [8]:
%%time
conll_reader = Conll2003DatasetReader()
train_conll = ensure_list(conll_reader.read(path_train))
valid_conll = ensure_list(conll_reader.read(path_valid))
test_conll = ensure_list(conll_reader.read(path_test))

14041it [00:01, 11745.55it/s]
3250it [00:00, 8785.82it/s]
3453it [00:00, 21923.31it/s]

CPU times: user 1.65 s, sys: 70.5 ms, total: 1.72 s
Wall time: 1.73 s


In [9]:
all_conll = train_conll + valid_conll + test_conll
len(all_conll), len(train_conll), len(valid_conll), len(test_conll)

(20744, 14041, 3250, 3453)

In [10]:
tags = set()
tokens = set()

max_sent_len = 0
for instance in all_conll[: ]:
    if len(instance['tokens']) >  max_sent_len:
        max_sent_len = len(instance['tokens'])
    tags.update(instance['tags'])
    tokens.update(instance['tokens'])
    
max_sent_len

124

In [11]:
max_token_len = max([len(token.text) for token in tokens])
max_token_len

61

In [12]:
chars = set()
for token in tokens:
    chars.update(token.text)
    
len(chars)

85

In [0]:
tag2id = {tag: num for num, tag in enumerate(['<NOTAG>'] + list(tags))}
char2id = {char: num+1 for num, char in enumerate(chars)}
id2char = {item[1]: item[0] for item in char2id.items()}

In [14]:
%%time

train_inputs, train_targets = tag_corpus_to_tensor(train_conll, char2id, tag2id, max_sent_len, max_token_len)
valid_inputs, valid_targets = tag_corpus_to_tensor(valid_conll, char2id, tag2id, max_sent_len, max_token_len)
test_inputs, test_targets = tag_corpus_to_tensor(test_conll, char2id, tag2id, max_sent_len, max_token_len)

100%|██████████| 3453/3453 [00:01<00:00, 1842.29it/s]

CPU times: user 12.3 s, sys: 568 ms, total: 12.8 s
Wall time: 12.8 s


In [0]:
train_dataset = TensorDataset(train_inputs, train_targets)
valid_dataset = TensorDataset(valid_inputs, valid_targets)
test_dataset = TensorDataset(test_inputs, test_targets)

In [0]:
models_path = '/content/cnn_crf_nertagger/models/best_model.pth'

In [17]:
try:
    del model    
    collect()
except:
    print('no model')
finally:
    torch.cuda.empty_cache()


no model


In [18]:
torch.cuda.empty_cache()
model = NERTaggerModel(len(char2id), len(tag2id), tag2id, embedding_size=32,
                                              single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.2, dilation=[1,1,1]),
                                              context_backbone_kwargs=dict(layers_n=5, kernel_size=3, dropout=0.2, dilation=[1, 1, 2, 3, 1]))
print('Количество параметров', sum(np.product(t.shape) for t in model.parameters()))

Количество параметров 28069


In [0]:
# try:
#     model.load_state_dict(torch.load(models_path))
# except:
#     print('no model')

In [20]:
(best_val_loss,
 best_model) = train_eval_loop(model,
                               train_dataset,
                               valid_dataset,
                               lr=2.5e-2,
                               epoch_n=200,
                               batch_size=256,
                               device=device,
                               early_stopping_patience=8,
                               l2_reg_alpha = 1e-6,
                               max_batches_per_epoch_train=100,
                               max_batches_per_epoch_val=250,
                               lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(
                                                                                     optim, patience=3,
                                                                                     factor=0.2,
                                                                                     threshold=1e-4,
                                                                                     verbose=True),
                               verbose_batch=False)
 
torch.save(best_model.state_dict(), models_path)

Эпоха 0
Эпоха: 55 итераций, 15.68 сек
Среднее значение функции потерь на обучении 13.690856118635697
Среднее значение функции потерь на валидации 5.635078026698186
Новая лучшая модель!

Эпоха 1
Эпоха: 55 итераций, 15.61 сек
Среднее значение функции потерь на обучении 4.116903274709528
Среднее значение функции потерь на валидации 3.1206105855795054
Новая лучшая модель!

Эпоха 2
Эпоха: 55 итераций, 15.62 сек
Среднее значение функции потерь на обучении 2.949314407868819
Среднее значение функции потерь на валидации 2.493960609802833
Новая лучшая модель!

Эпоха 3
Эпоха: 55 итераций, 15.58 сек
Среднее значение функции потерь на обучении 2.522727528485385
Среднее значение функции потерь на валидации 2.2361261936334462
Новая лучшая модель!

Эпоха 4
Эпоха: 55 итераций, 15.57 сек
Среднее значение функции потерь на обучении 2.2990297707644376
Среднее значение функции потерь на валидации 2.046815743813148
Новая лучшая модель!

Эпоха 5
Эпоха: 55 итераций, 15.46 сек
Среднее значение функции потерь н

In [31]:

model.load_state_dict(torch.load(models_path))

<All keys matched successfully>

## Проверки

In [0]:
id2tag = {item[1]: item[0] for item in tag2id.items()}
UNIQUE_TAGS = [id2tag[i] for i in range(len(tag2id))]


### Проверка - train

In [23]:
train_pred = predict_with_model(model, train_dataset)
# train_loss = F.cross_entropy(torch.tensor(train_pred),
                            #  torch.tensor(train_targets))
# print('Среднее значение функции потерь на обучении', float(train_loss))

print(classification_report(train_targets.view(-1), train_pred.reshape(-1), target_names=UNIQUE_TAGS))
print()

100%|██████████| 439/438.78125 [00:14<00:00, 31.20it/s]


              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1537463
      I-MISC       0.70      0.88      0.78      4556
       I-ORG       0.72      0.92      0.81     10001
           O       1.00      0.97      0.99    169578
       I-PER       0.93      0.95      0.94     11128
      B-MISC       1.00      0.14      0.24        37
       B-ORG       0.00      0.00      0.00        24
       B-LOC       0.00      0.00      0.00        11
       I-LOC       0.90      0.91      0.91      8286

    accuracy                           1.00   1741084
   macro avg       0.69      0.64      0.63   1741084
weighted avg       1.00      1.00      1.00   1741084




### Проверка - valid

In [24]:
valid_pred = predict_with_model(model, valid_dataset)
# valid_loss = F.cross_entropy(torch.tensor(valid_pred),
#                             torch.tensor(valid_targets))
# print('Среднее значение функции потерь на валидации', float(valid_loss))

# print(classification_report(valid_targets.view(-1), valid_pred.reshape(-1), target_names=UNIQUE_TAGS))

100%|██████████| 102/101.5625 [00:03<00:00, 28.78it/s]


### Проверка - test

In [25]:
test_pred = predict_with_model(model, test_dataset)
# test_loss = F.cross_entropy(torch.tensor(test_pred),
#                             torch.tensor(test_targets))
# print('Среднее значение функции потерь на тесте', float(test_loss))
print(classification_report(test_targets.view(-1), test_pred.reshape(-1), target_names=UNIQUE_TAGS))

100%|██████████| 108/107.90625 [00:03<00:00, 30.56it/s]


              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00    381737
      I-MISC       0.53      0.77      0.62       909
       I-ORG       0.60      0.80      0.68      2491
           O       1.00      0.96      0.98     38323
       I-PER       0.83      0.87      0.85      2773
      B-MISC       0.00      0.00      0.00         9
       B-ORG       0.00      0.00      0.00         5
       B-LOC       0.00      0.00      0.00         6
       I-LOC       0.80      0.81      0.81      1919

    accuracy                           0.99    428172
   macro avg       0.53      0.58      0.55    428172
weighted avg       0.99      0.99      0.99    428172



## Применение теггера

In [0]:
!python -m spacy download en_core_web_sm > /dev/null

In [0]:

ner_tagger = NERTagger(model, char2id, UNIQUE_TAGS, max_sent_len, max_token_len)

In [0]:
nlp = spacy.load('en_core_web_sm')


In [0]:
test_sentences = [
    'Mr Trump said Mr Linick no longer had his full confidence and that he would be removed in 30 days.',
    'Mr Linick had begun investigating Secretary of State Mike Pompeo for suspected abuse of office, reports say.',
    'Democrats say Mr Trump is retaliating against public servants who want to hold his administration to account.',
    'Donald Trump, who is campaigning for re-election in November, has stepped up his attacks on China in recent weeks, blaming it for the spread of Covid-19.'
]
test_sentences_tokenized = tokenize_corpus(test_sentences)
# test_sentences_tokenized = [[token.text for token in nlp.tokenizer(sent) ] for sent in test_sentences]

In [30]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, ner_tagger(test_sentences)):
    print()
    assert len(sent_tokens) == len(sent_tags)
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))

1it [00:00, 116.36it/s]                  


Mr-I-ORG Trump-I-ORG said-O Mr-I-PER Linick-I-PER no-O longer-O had-O his-O full-O confidence-O and-O that-O he-O would-O be-O removed-O in-O 30-O days-O .-O

Mr-I-ORG Linick-I-ORG had-O begun-O investigating-O Secretary-O of-O State-I-PER Mike-I-PER Pompeo-I-PER for-O suspected-O abuse-O of-O office-O ,-O reports-O say-O .-O

Democrats-I-MISC say-O Mr-I-PER Trump-I-PER is-O retaliating-O against-O public-O servants-O who-O want-O to-O hold-O his-O administration-O to-O account-O .-O

Donald-I-PER Trump-I-PER ,-O who-O is-O campaigning-O for-O re-O --O election-O in-O November-O ,-O has-O stepped-O up-O his-O attacks-O on-O China-I-LOC in-O recent-O weeks-O ,-O blaming-O it-O for-O the-O spread-O of-O Covid-19-I-MISC .-O
